In [ ]:
#importing the neccesary libraries
from math import sqrt,log2
import numpy as np
import random
import collections   
from sortedcontainers import SortedDict


In [ ]:
#dummy initialisations
NO_OF_USERS = 100
NO_OF_BROADCAST_SERVERS = 10
BW = 10 #bandwidth 
alpha = 2.5# txf
NOISE = 0.0005
DISTANCE_THRES = 100


In [ ]:
#Defining User and Bandwidth-Server classes
class User:
    def __init__(self,uID,loc):
        self.preference_list =[]
        self.uID = uID
        self.X = loc[0]
        self.Y = loc[1]
        self.target = None
class BS :
    def __init__(self,bID,loc):
        self.preference_list = []
        self.bID = bID
        self.X = loc[0]
        self.Y = loc[1]
        self.associatedUsers = set()  


In [ ]:
random.seed(1)
x_coord = random.sample(range(1,500),NO_OF_USERS+NO_OF_BROADCAST_SERVERS)
random.seed(2)
y_coord = random.sample(range(1,500),NO_OF_USERS+NO_OF_BROADCAST_SERVERS)


#the users have been id from 1 to 100
#the BSs have been id-ed from 101 to 110
Users = [User(i+1,(x_coord[i],y_coord[i])) for i in range(100)]
BSs = [BS(i+100,(x_coord[i],y_coord[i])) for i in range(100,110)]


In [ ]:
#this matrix stores the distance between all possible pairs of servers and users with shape  (10,100)
distance_Mat =[sqrt((Users[j].X -BSs[i].X )**2 + (Users[j].Y -BSs[i].Y )**2) for i in range(0,10) for j in range (0,100)]
distance_Mat = (np.array(distance_Mat)).reshape(10,100)
distance_Mat[0:2][0:50]

array([[320.22492095, 192.31484602, 281.25611104, 226.10838109,
        247.30952266, 320.48868935, 254.82543044, 229.40139494,
        180.03610749, 196.08671551, 136.48809472, 169.54645381,
        107.04204781, 206.73654733, 153.60013021, 211.11608181,
        239.8853893 , 144.36758639, 353.61702448, 291.01546351,
        154.63828763, 127.91403363, 178.05897899,  42.44997055,
        134.27211177, 121.85236969, 322.62206992, 142.88806808,
        197.95201439, 144.41952777, 106.78014797, 216.94699814,
        164.47796205,  23.40939982, 254.02362095, 278.75616585,
        258.50338489, 329.60734215, 357.19182521, 274.33191575,
         53.15072906,  86.05230967, 281.42672226, 174.15510328,
        220.07725916,  73.76313442, 236.90715481,  73.92563831,
        183.19934498, 237.9936974 , 131.38112498, 289.92757716,
        293.59325605, 100.12492197, 264.6148144 , 180.02499826,
        161.44658559, 219.23731434, 291.2181313 ,  67.46851117,
        193.49418596, 290.66303515, 162.

In [ ]:
# #User preference list 

def preflistBuilderforBS():
    for i in range(10):
        pref_dict =  SortedDict()              #collections.OrderedDict()
        pref_list =[] 
        for j in range(100):
            a = distance_Mat[i][j]
            if a<= DISTANCE_THRES:
                pref_dict[a] = j+1
        for key,value in pref_dict.items():
            pref_list.append(value)
        BSs[i].preference_list = pref_list
        
    
            

In [ ]:
# def preflistBuilderforBS():
#     for i in range(10):
#         pref_dict =  SortedDict()                #collections.OrderedDict()
#         pref_list =[] 
#         for j in range(100):
#             a = distance_Mat[i][j]
#             pref_dict[a] = j+1
#         for key,value in pref_dict.items():
#             pref_list.append(value)
#         BSs[i].preference_list = pref_list


In [ ]:
#BS preference list

"""B = BW/no of user * (1+log2(SINR))
SINR = rx / sum of interferences + noise """



def preflistBuilderforUsers(n:int):   #n is no of users  in BS
    rx = alpha/(distance_Mat**2)
    interactions= np.zeros((10,100))
    for i in range(10):
        for j in range(100):
            interactions[i][j] = np.sum([rx[p][j]for p in range(10) if p!=i])
       
    for j in range(100):
        pref_dict =SortedDict()
        pref_list =[] 
        for i in range(10):
            a = (BW/n)*log2(1+ (rx[i][j]/(NOISE + interactions[i][j])))
            pref_dict[a]=i+1
            
        for key,value in pref_dict.items():
            pref_list.append(value)
        pref_list.reverse()
        Users[j].preference_list = pref_list
          


In [ ]:
# def bitrate(n:int,router:int,user:int):
#   rx = alpha/(distance_Mat**2)
#   interactions= np.zeros((10,100))
#   for i in range(10):
#       for j in range(100):
#           interactions[i][j] = np.sum([rx[p][j]for p in range(10) if p!=i])
#   a = (BW/n)*log2(1+ (rx[router][user]/(NOISE + interactions[router][user])))
#   return a
  
    


# n1 is the number of users we have right now
# n2 is the number of routers we have 
# r is the ratio n1 : n2


In [ ]:
# n1 = NO_OF_USERS
# n2 = NO_OF_BROADCAST_SERVERS
# r = int(n1/n2)

# list1=[]
# list2=[]
# preflistBuilderforUsers(r)
# preflistBuilderforBS()

# for i in range(n1):
#   list1.append(Users[i].preference_list)
# for i in range(n2):
#   list2.append(BSs[i].preference_list)

# print(list1)
# print(list2)

# user_start=1
# router_start=101

In [ ]:
# procedure P R O P O S A L(i):
#     if Usercounter[i] <=  len(preflist[i]) :
#         j : = malecounter[i]
#         malecounter[i]++
#         //check if i is present in the list  of j and if j is allowed i.e.,space is available

#             if  j is free :
#                 filled[j]++
#                 marriage[j] := i;
                
#             else :
                
#                 if preflistBS[j, marriage[j]] > fc[j,i] 
#                     l = marriage[j]; 
#                     marriage[j] := i;
#                     filled[i]++
#                     filled[l]--
#                     PROPOSAL(l)
#                 else 
#                     P R O P O S A L( i ) 


# shouldpropose =[] //true for all
# seatsineachlist = []
# filledtill = []10 bs will initially have 0]
# for i in range(max(length of all seatsineachlist)):
#     for j in range(10):
#         if(shouldpropose withtrue)
#             propose(j);
#make all the shouldpropose false of those base stations who are full or didnot get engaged to anyone in this iteration


#     integer j ; count : = count + 1 ;
#     j : = malecounter[i] ; malecounter[i] : = j + 1 ;
#     R E F U S A L ( i , malechoice[i,j])
# p r o c e d u r e  R E F U S A L ( i : man , j :woman ) :















In [ ]:
listUsers=[]
listBSs=[]
r=NO_OF_USERS/NO_OF_BROADCAST_SERVERS

preflistBuilderforUsers(r)
preflistBuilderforBS()

for i in range(NO_OF_USERS):
  listUsers.append(Users[i].preference_list)
for i in range(NO_OF_BROADCAST_SERVERS):
  listBSs.append(BSs[i].preference_list)





userFree =[True for i in range(NO_OF_USERS)]   #keeps track of users who are free
userCount = [0 for i in range(NO_OF_USERS)] 


BSallowedForIteration=[True for i in range(NO_OF_BROADCAST_SERVERS)]
BSmaxCapacity = [len(listBSs[i]) for i in range(NO_OF_BROADCAST_SERVERS)]
BSprevcurrFilled = [0 for i in range(NO_OF_BROADCAST_SERVERS)] 
BScurrFilled =   [0 for i in range(NO_OF_BROADCAST_SERVERS)]    




marriage=[]   
for i in range(NO_OF_BROADCAST_SERVERS):
    marriage.append([])



satisfaction_rate =[0 for i in range(NO_OF_USERS)]
print(BSmaxCapacity )                    
                     
print(listUsers)

[18, 12, 12, 9, 16, 7, 7, 9, 19, 10]
[[5, 8, 4, 9, 1, 2, 10, 6, 7, 3], [8, 4, 2, 5, 1, 9, 7, 3, 10, 6], [4, 8, 2, 7, 5, 1, 9, 3, 10, 6], [4, 8, 2, 7, 1, 5, 9, 3, 10, 6], [3, 1, 9, 7, 2, 6, 10, 5, 4, 8], [6, 10, 9, 3, 1, 5, 2, 7, 8, 4], [6, 10, 3, 9, 1, 5, 2, 7, 8, 4], [10, 6, 9, 5, 1, 3, 2, 8, 7, 4], [5, 8, 4, 2, 1, 9, 7, 3, 10, 6], [3, 1, 9, 7, 2, 5, 10, 6, 4, 8], [5, 9, 2, 1, 8, 4, 7, 10, 3, 6], [5, 8, 2, 4, 1, 9, 7, 10, 3, 6], [2, 1, 7, 9, 8, 4, 5, 3, 10, 6], [5, 8, 4, 2, 9, 1, 7, 10, 3, 6], [3, 1, 7, 9, 2, 5, 4, 8, 10, 6], [10, 6, 9, 1, 3, 5, 2, 7, 8, 4], [5, 10, 9, 6, 1, 2, 8, 3, 4, 7], [3, 9, 1, 10, 6, 2, 5, 7, 8, 4], [5, 8, 10, 9, 1, 4, 2, 6, 7, 3], [3, 1, 7, 9, 2, 6, 10, 5, 4, 8], [7, 2, 1, 4, 9, 8, 3, 5, 10, 6], [5, 9, 1, 2, 8, 4, 10, 7, 3, 6], [3, 9, 6, 1, 10, 5, 2, 7, 8, 4], [1, 9, 3, 2, 7, 5, 8, 4, 10, 6], [2, 7, 1, 4, 8, 9, 5, 3, 10, 6], [3, 1, 7, 9, 2, 5, 4, 8, 10, 6], [5, 10, 9, 1, 6, 8, 2, 4, 7, 3], [2, 4, 8, 7, 1, 9, 5, 3, 10, 6], [5, 8, 4, 2, 9, 1, 7, 10, 3, 6], [5, 9

In [ ]:
def proposal(user,itern):
    if userCount[user] < NO_OF_BROADCAST_SERVERS:
        pref_no  = userCount[user] 
        BStoPropose = listUsers[user][pref_no]
        BStoPropose=BStoPropose-1
        userCount[user] = userCount[user]+1
        #print(user+1,(user+1) in listBSs[BStoPropose])
        if BSallowedForIteration[BStoPropose] == True and (user+1) in listBSs[BStoPropose] :
            if BScurrFilled[BStoPropose] < itern:
                marriage[BStoPropose].append(user+1)
                BScurrFilled[BStoPropose]=1+BScurrFilled[BStoPropose]
                userFree[user]=False
                satisfaction_rate[user] = (itern - 1)*10 + (pref_no+1)


            else:
                if listBSs[BStoPropose].index(marriage[BStoPropose][-1])> listBSs[BStoPropose].index(user+1) :
                    abanded =  marriage[BStoPropose][-1]
                    abanded=abanded-1
                    marriage[BStoPropose].pop()
                    marriage[BStoPropose].append(user+1)
                    userFree[user]==False
                    satisfaction_rate[user] = (itern - 1)*10 + (pref_no+1)


                    userFree[abanded]=True
                    satisfaction_rate[abanded] = 0
                    proposal(abanded,itern)
                else :
                    proposal(user,itern)
        

In [ ]:


for i in range(max(BSmaxCapacity)):
    for j in range(NO_OF_USERS):
        if userFree[j]==True :
            proposal(j,i+1)
    # for x in range(NO_OF_BROADCAST_SERVERS):
    #     print("Iteration ",i+1 , marriage[x])
        

    preflistBuilderforUsers(i+1)
    listUsers = []
    for x in range(NO_OF_USERS):
        listUsers.append(Users[x].preference_list)


    for x in range(NO_OF_BROADCAST_SERVERS):
        if BScurrFilled[x]==BSprevcurrFilled[x] or BScurrFilled[x] == BSmaxCapacity[x]  :
            BSallowedForIteration[x] = False
        BSprevcurrFilled[x]=BScurrFilled[x]


    
    
    





In [ ]:
for i in range(NO_OF_BROADCAST_SERVERS):
    print(marriage[i])

[68, 88]
[13]
[49]
[4]
[22]
[6, 35]
[21]
[2, 4]
[88, 68]
[35]


In [ ]:
satisfaction_rate

[0,
 1,
 0,
 12,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 12,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 12,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 12,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

# this following function allows a router to have r users connected with it
# it also assumes the preferences of routers available are correct


In [ ]:



bitrate_calculator=[-1 for i in range(n1)]
satisfaction_rate=[0 for i in range(n1)]


def stable(list1 ,list2,ratio):
  # list of patners of all routers
  
  #all users are free right now; i.e while initialising
  uNotFree = [False for i in range(n1)]
  #no of users who are still not married
  freeCount=n1

  rPatner=[]
  for i in range(n2):
    rPatner.append([])

  temp=1
  while (freeCount > 0):
    #this loop finds a user who is free
    u = 0
    while (u < n1):
      if (uNotFree[u]==False):
        break
      u+=1
    i = 0
    #this loop checks for each and every router in the list of user 'u'
    u+=user_start
    while i < n2 and uNotFree[u-user_start]==False:
      r = list1[u-user_start][i]
    
      if (len(rPatner[r-router_start]) < ratio):
        if (u in rPatner[r-router_start]):
          break
        else :
          rPatner[r-router_start].append(u)
          uNotFree[u-user_start]=True
          # satisfaction_rate[u-user_start]=(temp)*10+list1[u-user_start].index(r)
          freeCount-=1
          preflistBuilderforUsers(len(rPatner[r-router_start]))
          bitrate_calculator[u-user_start]=bitrate(len(rPatner[r-router_start]),r-router_start,u-user_start)
        # here you need to update the preference list of the users 
        # and hence dyanamically do it
      else :
        # find if there exists some patner whose priority is lesser than 'u'
        index_of_router=r-router_start
        index_u = list2[index_of_router].index(u)
        for x in range(ratio):
          curr = rPatner[index_of_router][x]
          index_of_curr_in_preferences = list2[index_of_router].index(curr)
          if (index_of_curr_in_preferences > index_u):
            uNotFree[curr-user_start]=False
            bitrate_calculator[curr-user_start]=-1
            satisfaction_rate[curr-user_start]=0
            rPatner[index_of_router][x]=u
            uNotFree[u-user_start]=True
            # satisfaction_rate[u-user_start]=(temp)*10+list1[u-user_start].index(r)
            bitrate_calculator[u-user_start]=bitrate(len(rPatner[r-router_start]),r-router_start,u-user_start);
            break

          
         
      i+=1
    satisfaction_rate[u-user_start]=(temp)*10+list1[u-user_start].index(r)
    #print("Satisfaction rate : ", temp , " ",list1[u-user_start].index(r))
    temp+=1


  return rPatner          


      
  

In [ ]:
def stable_marriage(list1,list2,ratio):
  #list1 contains the preference of users
  #list2 contains the preference of servers
  server_Patner=[]
  for i in range(n2):
    server_Patner.append([])
  user_free=[True for i in range(n1)]
  free_count=n1
  iteration_number=1
  while (free_count > 0):
    user__=0
    for i in range(n1):
      if (user_free[user__]==True):
        break
      user__+=1
    
    user_name=user__+user_start
    i=0
    while i < n2 and user_free[user__]==True:
        r = list1[user__][i]

        #no of users which can be married to a server at a given time
        #for the first iteration itll be 1, after that 2 and so on
        if (len(server_Patner[r-router_start]) < min(iteration_number,ratio) ):
          if (user_name in server_Patner[r-router_start]):
            break
          else :
            server_Patner[r-router_start].append(user_name)
            user_free[user__]=False
            free_count-=1
        else :
          # find if there exists some patner whose priority is lesser than 'u'
          index_of_router=r-router_start
          index_u = list2[index_of_router].index(user_name)
          for x in range(len(server_Patner[r-router_start])):
            curr = server_Patner[index_of_router][x]
            index_of_curr_in_preferences = list2[index_of_router].index(curr)
            if (index_of_curr_in_preferences > index_u):
              user_free[curr-user_start]=True
              server_Patner[index_of_router][x]=user_name
              user_free[user__]=False
              break
        i+=1
    iteration_number+=1        
    


  return server_Patner



In [ ]:
answer=stable(list1,list2,r)

IndexError: ignored

In [ ]:
answer

[[87, 34, 24, 41, 60, 68, 26, 83, 63, 71],
 [13, 25, 28, 46, 69, 72, 79, 75, 98, 38],
 [76, 10, 73, 15, 51, 56, 18, 31, 49, 5],
 [3, 4, 36, 53, 62, 67, 99, 70, 1, 59],
 [97, 77, 11, 80, 96, 82, 12, 22, 14, 61],
 [6, 7, 39, 47, 52, 27, 17, 92, 23, 19],
 [21, 32, 44, 55, 50, 65, 78, 81, 20, 95],
 [2, 91, 89, 29, 90, 58, 37, 45, 9, 94],
 [88, 93, 48, 100, 42, 66, 54, 85, 64, 30],
 [8, 84, 16, 35, 86, 33, 40, 43, 74, 57]]

In [ ]:
satisfaction_rate

In [ ]:
bitrate_calculator

In [ ]:
def index_2d(myList, v):
    for i, x in enumerate(myList):
        if v in x:
            return (i, x.index(v))

In [ ]:
final_answer_dict={}
for i in range(n1):
  final_answer_dict[i+user_start]=[]
for i in range(n1):
  final_answer_dict[i+user_start].append(bitrate_calculator[i])
  final_answer_dict[i+user_start].append(satisfaction_rate[i])
  r=index_2d(answer,i+user_start)
  final_answer_dict[i+user_start].append(r[0]+router_start)

In [ ]:
final_answer_dict

In [ ]:
from prettytable import PrettyTable as pt

In [ ]:
t=pt(['User_ID','Bitrate_Received_For_Marrying','Satisafcation_Rate','Band-width_Server']) 

In [ ]:
for i in range(n1):
  list_temp=[]
  list_temp.append(i+user_start)
  list_temp.append(final_answer_dict[i+user_start][0])
  list_temp.append(final_answer_dict[i+user_start][1])
  list_temp.append(final_answer_dict[i+user_start][2])
  t.add_row(list_temp)
  list_temp.clear()

In [ ]:
t

In [ ]:
print(t)

+---------+-------------------------------+--------------------+-------------------+
| User_ID | Bitrate_Received_For_Marrying | Satisafcation_Rate | Band-width_Server |
+---------+-------------------------------+--------------------+-------------------+
|    1    |      0.05472085048464461      |        2102        |        1004       |
|    2    |       7.1494653974286315      |         20         |        1008       |
|    3    |       7.154874550909796       |         30         |        1004       |
|    4    |       4.5461429623057725      |         40         |        1004       |
|    5    |       0.4573802897111907      |        1180        |        1003       |
|    6    |       14.91391488990507       |         60         |        1006       |
|    7    |       2.0254540500853495      |         70         |        1006       |
|    8    |       21.885062136602098      |         80         |        1010       |
|    9    |      0.31056618189147867      |        1401        | 

In [ ]:
def stable_2(list1,list2,start_user,server_patner):
  

  server_free=[True for i in range(n2)]
  
  
  for user__ in range(n1):
    i=0
    user_index_in_list2=user__+start_user
    while (i < n2 and user_free[user__]):
      server=list1[user_index_in_list2][i]
      server_index=server-router_start

      if (server_free[server_index]):
        server_patner[server_index].append(user__+start_user+user_start)
        user_free[user__]=False
        server_free[server_index]=False
        #free_count-=1
      else:
        s=len(server_patner[server_index])
        curr_patner=server_patner[server_index][s-1]

        index_curr=list2[server_index].index(curr_patner)
        index_user=list2[server_index].index(user__+start_user+user_start)

        if (index_user < index_curr):
          server_patner[server_index][s-1]=user__+start_user+user_start
          user_free[user__]=False
          user_free[curr_patner-user_start-start_user]=True
      i+=1
    


      
  


In [ ]:
def stable_marriage(list1,list2,ratio):
  server_patner=[]
  for i in range(n2):
    server_patner.append([])

  for i in range(ratio):
    initial=0

    stable_2(list1,list2,initial,server_patner)

  return server_patner



    



In [ ]:
user_free=[True for i in range(n1)]
free_count=n1
